# Convex Nonparametric Least Square (`CNLS`)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 26, 2020

## Production function estimation

Hildreth (1954) was the first to consider nonparametric regression subject to monotonicity and concavity constraints in the case of a single input variable $x$. Kuosmanen (2008) extended Hildreth’s approach to the multivariate setting with a vector-valued $\bf{x}$, and coined the term convex nonparametric least squares (`CNLS`) for this method. `CNLS` builds upon the assumption that the true but unknown production function $f$ belongs to the set of continuous, monotonic increasing and globally concave functions, imposing exactly the same production axioms as standard DEA. 

The multivariate `CNLS` formulation is defined as:

\begin{align*}
& \underset{\alpha, \beta, \varepsilon} {min} \sum_{i=1}^n\varepsilon_i^2 \\
& \text{s.t.} \\
&  y_i = \alpha_i + \beta_i^{'}X_i + \varepsilon_i \quad \forall i \\
&  \alpha_i + \beta_i^{'}X_i \le \alpha_j + \beta_j^{'}X_i  \quad  \forall i, j\\
&  \beta_i \ge 0 \quad  \forall i \\
\end{align*}

where $\alpha_i$ and $\beta_i$ define the intercept and slope parameters of tangent hyperplanes that characterize the estimated piece-wise linear frontier. $\varepsilon_i$ denotes the CNLS residuals. The first constraint can be interpreted as a multivariate regression equation, the second constraint imposes convexity, and the third constraint imposes monotonicity.

## Data source

Regulation of Finnish electricity distribution firms
  + Data set: https://github.com/ds2010/StoNED-Python/tree/master/Data
  + Data description: https://github.com/ds2010/StoNED-Python/blob/master/Data/data_description.ipynb

## Example

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import CNLS

In [3]:
# import data
df = pd.read_excel('data.xlsx')

# output
y = df['Energy']
y = y.to_numpy()
y = y.tolist()

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)
x = x.tolist()

In [4]:
# define and solve the CNLS model

crt = "addi"
func = "prod"
pps = "vrs"

model = CNLS.cnls(y, x, crt, func, pps)

from pyomo.environ import *
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [5]:
# display the estimates (alpha, beta, and residual)
model.a.display()
model.b.display()
model.e.display()

a : alpha
    Size=89, Index=i
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :  None :  -22.70516387743662 :  None : False : False :  Reals
      1 :  None :  -22.70521277764463 :  None : False : False :  Reals
      2 :  None :  -22.70510132657636 :  None : False : False :  Reals
      3 :  None :    33.5498045426187 :  None : False : False :  Reals
      4 :  None :  -22.65700498261703 :  None : False : False :  Reals
      5 :  None :  -17.84513092549913 :  None : False : False :  Reals
      6 :  None :    -22.705149179287 :  None : False : False :  Reals
      7 :  None : -17.845217469696976 :  None : False : False :  Reals
      8 :  None : -22.705123983910507 :  None : False : False :  Reals
      9 :  None :   -17.8454509809616 :  None : False : False :  Reals
     10 :  None :    110.961103190736 :  None : False : False :  Reals
     11 :  None :    5790.15905337689 :  None : False : False :  Reals
     12 :  None :  -22.70506356116528 :  None 

In [6]:
# retrive the alpha
val = list(model.a[:].value)
alpha = np.asarray(val)
alpha

array([-2.27051639e+01, -2.27052128e+01, -2.27051013e+01,  3.35498045e+01,
       -2.26570050e+01, -1.78451309e+01, -2.27051492e+01, -1.78452175e+01,
       -2.27051240e+01, -1.78454510e+01,  1.10961103e+02,  5.79015905e+03,
       -2.27050636e+01, -1.78452395e+01,  2.02149568e+03, -1.78449353e+01,
       -2.27051916e+01, -2.27051437e+01, -2.27051285e+01, -1.78449534e+01,
       -1.87883528e+01, -2.10761437e+01, -2.10068678e+07, -1.78448404e+01,
       -2.27051813e+01, -2.27050767e+01, -2.27050807e+01,  3.35498051e+01,
       -2.27051782e+01, -1.78452306e+01, -2.27051333e+01, -1.16573113e+02,
       -1.78450153e+01, -1.78451733e+01, -1.78640852e+01, -1.78449274e+01,
       -1.78450012e+01, -2.27051916e+01, -2.27052114e+01, -1.78448319e+01,
       -1.78449199e+01, -2.27052146e+01,  3.35498056e+01, -2.27051526e+01,
       -2.27051233e+01, -2.26570196e+01, -1.78448829e+01, -2.27052142e+01,
       -1.78449274e+01,  3.35497932e+01, -2.26570161e+01, -1.78450066e+01,
       -2.27051106e+01, -

In [7]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)
eps

array([  -2.80719914,    1.35674283,  -22.18826566, -350.98017376,
        -13.8278582 ,  101.10740333,  -28.89452581,  -13.94195993,
         -0.8432533 ,   56.9804638 ,  285.45176794,  679.40966853,
        -20.16880962,  -69.89436819,   10.5090532 ,   74.66403408,
         -6.61022336,  -30.09657804,  -40.129151  ,  -27.68777946,
         48.83399892,   87.07326581,   22.24746582,   24.02219238,
         -2.19972735,  -22.79188447,  -37.81260266, -351.13253835,
         66.79558941,  -21.60033681,   -8.96190813,  216.17804228,
         37.70979481,  -31.6892427 ,  -23.71271687, -201.25886075,
        -69.21385864,    6.23651531,    3.64667393,  -74.57617583,
         -1.93710202,  -12.08866359, -236.80879387,    6.57187285,
         11.66887944,  -20.11491224,  -67.22255097,   -5.38466347,
        -55.88414275,  265.43932874,    1.79083646,  -17.55743156,
          4.67766135,   38.89751789,   -2.91801841,  349.45117483,
        164.05227227,   35.01811299,   28.44964606,  -99.02498

In [8]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:, :].value)
beta = np.asarray([i + tuple([j]) for i, j in zip(ind, val)])

beta = pd.DataFrame(beta, columns=['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2']
beta

,b1,b2
Name,,
0.0,0.135503,1.129624e-02
1.0,0.135503,1.129593e-02
2.0,0.135503,1.129630e-02
3.0,0.132350,-5.610955e-07
4.0,0.146742,2.253287e-03
...,...,...
84.0,0.134062,8.143116e-03
85.0,0.135503,1.129659e-02
86.0,0.135503,1.129631e-02
